## Bitcoin Bandit's 

## Цифровой прорыв. Сеpон ИИ. Кейс Rutube - Распознавание именованных сущностей в названиях и описаниях к видео

### Загрузка всех необходимых библиотек

In [6]:
import pandas as pd
import json
import logging
import numpy as np
import torch
from razdel import tokenize
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers.optimization import Adafactor, AdafactorSchedule
from transformers import DataCollatorForTokenClassification
from torch import optim
from transformers.trainer import logger as noisy_logger
from sklearn.metrics import confusion_matrix
from transformers import pipeline
from tqdm.notebook import tqdm

### Загрузка данных из CSV файла

In [7]:
# считаем данные
data = pd.read_csv("dataset/ner_data_train.csv")

In [8]:
data.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"{""label"":""организация""\,""offset"":196\,""length""..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"{""label"":""название проекта""\,""offset"":12\,""len..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"{""label"":""не найдено""\,""offset"":162\,""length"":..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."


In [9]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать,
# удалить лишние '\' например, преобразовать из str в список dict-ов
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [10]:
df.head(3)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"[{'label': 'локация', 'offset': 26, 'length': ..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"[{'label': 'организация', 'offset': 196, 'leng..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"[{'label': 'название проекта', 'offset': 12, '..."


In [11]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах

    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

### Обработка датасете и разобьение на train и test

In [12]:
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [13]:
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
4705,"[<, НАЗВАНИЕ, :, >, А, ., Ч, ., Бхактиведанта, Свами, Прабхупада, =, Биографический, фильм, <, ОПИСАНИЕ, :, >, ⚠️, ВНИМАНИЕ⚠️, Друзья, ,, если, вы, готовы, поддержать, наш, канал, ,, вот, несколько, способов, :, 💰, Яндекс, Деньги, :, 4100, 1197, 2379, 809, 💰, 💰, QIWI, кошелек, :, qiwi, ., com, /...","[O, O, O, O, B-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
4073,"[<, НАЗВАНИЕ, :, >, Перевал, Дятлова, ., Колеватов, =, ключ, к, разгадке, произошедшего, <, ОПИСАНИЕ, :, >, Перевал, Дятлова, ., Колеватов, ключ, к, разгадке, произошедшего, автор, считает, ,, что, именно, Колеватов, ,, является, важным, звеном, в, разгадке, тайны, ,, произошедшего, на, перевале...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-персона, I-персона, I-персона, O, O, O, O, O, O, O, O, O, O]"
3068,"[<, НАЗВАНИЕ, :, >, Скандальные, разводы, звезд, и, долги, по, алиментам, |, «, Звезды, сошлись, », ., Выпуск, от, 11, сентября, 2022, года, <, ОПИСАНИЕ, :, >, Развод, и, девичья, фамилия, :, почему, от, суда, и, раздела, имущества, знаменитостей, не, спасает, даже, брачный, контракт, ?, На, как...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-название проекта, I-название проекта, O, O, O, B-Дата, I-Дата, I-Дата, I-Дата, I-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [14]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [15]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5137
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1285
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [16]:
model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 64

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='gpu')

In [17]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'Московский', 'международный', 'фестиваль', 'мира', '=', '89', 'и', 'Стас', 'Намин', '.', '«', 'Главный', 'день', '»', '<', 'ОПИСАНИЕ', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центральном', 'стадионе', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'международный', 'рок', 'фестиваль', '.', 'Концерт', 'собрал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'молодежь', 'тогда', 'увидела', 'и', 'услышала', 'признанных', 'мировых', 'кумиров', ':', 'Scorpions', ',', 'Bon', 'Jovi', ',', 'Ozzy', 'Osbourne', ',', 'Cinderella', ',', 'Motley', 'Crue', '.', 'Приезд', 'западных', 'звезд', 'казался', 'ожившей', 'фантастикой', '.', 'Фестиваль', 'стал', 'не', 'только', 'музыкальной', ',', 'но', 'и', 'важной', 'политической', 'акцией', '.', 'Вся', 'мировая', 'пресса', 'писала', 'о', 'том', ',', 'что', 'СССР', 'и', 'США', ',', 'долгое', 'время', 'находившиеся', 'в', 'состоянии', '"', 'холодной', 'войны', '"', ',', 'наконец', 'то', 'с

In [18]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'Н', '##АЗ', '##В', '##АН', '##И', '##Е', ':', '>', 'Московский', 'между', '##народный', 'фестиваль', 'мира', '=', '89', 'и', 'С', '##тас', 'На', '##мин', '.', '«', 'Главный', 'день', '»', '<', 'О', '##П', '##И', '##С', '##АН', '##И', '##Е', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центр', '##альном', 'стадион', '##е', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'между', '##народный', 'рок', 'фестиваль', '.', 'К', '##он', '##цер', '##т', 'со', '##брал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'м', '##оло', '##де', '##ж', '##ь', 'тогда', 'у', '##вид', '##ела', 'и', 'у', '##сл', '##ыш', '##ала', 'признан', '##ных', 'мир', '##овых', 'к', '##уми', '##ров', ':', 'Sc', '##orp', '##ions', ',', 'Bon', 'Jovi', ',', 'Oz', '##zy', 'Os', '##bour', '##ne', ',', 'Ci', '##nder', '##ella', ',', 'Mot', '##ley', 'C', '##rue', '.', 'При', '##езд', 'запад', '##ных', 'зв', '##езд', 'каз', '##ался', 'о', '##жив', '##шей', 'ф', '##ант', '##аст'

In [19]:
len(example["tags"]), len(tokenized_input["input_ids"])

(170, 274)

In [20]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 11, 12, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 19, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 43, 43, 43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 51, 51, 52, 53, 53, 53, 54, 55, 55, 55, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 62, 63, 64, 65, 65, 66, 66, 66, 67, 68, 68, 68, 69, 70, 70, 71, 71, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 77, 78, 78, 78, 78, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 88, 89, 90, 90, 90, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 104, 105, 106, 106, 106, 107, 108, 109, 110, 110, 110, 111, 112, 113, 114, 115, 116, 117, 117, 117, 118, 118, 118, 118, 119, 120, 121, 121, 122, 123, 124, 124, 125, 125, 126, 126, 127, 127, 127, 128, 128, 129, 130, 130, 130, 130, 131, 132, 132, 133, 133, 134, 135, 136, 137, 137, 137, 138, 139, 139, 139, 140, 141, 142, 143, 144, 144, 144, 

In [21]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

274 274


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [22]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 32, 293, 16218, 8117, 20795, 8759, 14386, 30, 34, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 32, 294, 3932, 8759, 3330, 20795, 8759, 14386, 30, 34, 294, 18147, 5972, 2386, 1928, 733, 22970, 613, 17565, 29484, 15479, 12122, 18398, 13334, 1, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 18, 282, 3200, 860, 20727, 12029, 4375, 29463, 2013, 2262, 18, 294, 15735, 29137, 320, 18029, 18398, 24952, 753, 705, 11316, 4307, 329, 22067, 12427, 283, 4297, 292, 29172, 3330, 1928, 705, 13790, 294, 22797, 1768, 10203, 761, 13951, 3243, 2535, 2641, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [24]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [25]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Специальный объект для удобного формирования батчей

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [27]:
metric = load_metric("seqeval")

C:\Users\Maxim\AppData\Local\Temp\ipykernel_24712\152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [28]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'бренд': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [40]:
!pip install numba
from numba import cuda
cuda.select_device(0)
cuda.close()

  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/cd/59/5dd8a3059997ec1daf6f9f7c9b1aef7f0a9e23e1334a5774eae65cae6fd0/numba-0.58.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for llvmlite<0.42,>=0.41.0dev0 from https://files.pythonhosted.org/packages/14/fe/d3a9c921a2adad2e9f24693754983f290e0dac9410666e802b9dba4d0218/llvmlite-0.41.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
    --------------------------------------- 0.0/2.6 MB 495.5 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.6 MB 1.0 MB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.6 MB 1.5 MB/s eta 0:00:02
   ----- ---------------------------------- 0.4/2.6 MB 1.8 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.6 MB 1.8 MB/s eta 0:00:02
   ------------ --------------------------- 0.


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none'
)

In [42]:
# что мы видим без дообучения модели
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
# optimizer = Adafactor(
#             model.parameters(),
#             scale_parameter=True, # If True, learning rate is scaled by root mean square
        # )
# lr_scheduler = AdafactorSchedule(optimizer)
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

  0%|          | 0/21 [00:00<?, ?it/s]

{'eval_loss': 2.6117429733276367,
 'eval_precision': 0.003958691910499139,
 'eval_recall': 0.012951908998761122,
 'eval_f1': 0.006063961612486487,
 'eval_accuracy': 0.6777954326413206,
 'eval_runtime': 99.6811,
 'eval_samples_per_second': 12.891,
 'eval_steps_per_second': 0.211}

In [32]:
noisy_logger.setLevel(logging.WARNING)

In [33]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [43]:
trainer.train()

  0%|          | 0/4050 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [47]:
trainer.evaluate()

{'eval_loss': 0.4255605638027191,
 'eval_precision': 0.3826612316203232,
 'eval_recall': 0.2960355895934227,
 'eval_f1': 0.3338201676403353,
 'eval_accuracy': 0.8927560068915987,
 'eval_runtime': 11.7335,
 'eval_samples_per_second': 109.515,
 'eval_steps_per_second': 3.494,
 'epoch': 20.0}

In [48]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Дата': {'precision': 0.4505978602894902,
  'recall': 0.5966666666666667,
  'f1': 0.5134456794550019,
  'number': 1200},
 'бренд': {'precision': 0.3333333333333333,
  'recall': 0.03117782909930716,
  'f1': 0.057022175290390706,
  'number': 866},
 'вид спорта': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 251},
 'видеоигра': {'precision': 0.0635593220338983,
  'recall': 0.020107238605898123,
  'f1': 0.030549898167006113,
  'number': 746},
 'команда': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 412},
 'лига': {'precision': 0.16666666666666666,
  'recall': 0.0070921985815602835,
  'f1': 0.013605442176870748,
  'number': 141},
 'локация': {'precision': 0.37008281573498963,
  'recall': 0.47587354409317806,
  'f1': 0.41636337166982096,
  'number': 3005},
 'модель': {'precision': 0.13793103448275862,
  'recall': 0.03278688524590164,
  'f1': 0.05298013245033113,
  'number': 488},
 'название проекта': {'precision': 0.29573420836751435,
  'recall': 0.22048929663608563,
  'f

In [50]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-Дата,B-бренд,B-вид спорта,B-видеоигра,B-команда,B-лига,B-локация,B-модель,B-название проекта,...,I-видеоигра,I-команда,I-лига,I-локация,I-модель,I-название проекта,I-организация,I-персона,I-сезон,I-серия
O,227844,471,33,0,9,0,0,1209,13,650,...,113,0,1,68,41,622,32,1307,0,0
B-Дата,294,847,0,0,0,0,0,23,0,2,...,1,0,0,0,0,2,0,0,0,0
B-бренд,735,0,28,0,3,0,0,31,14,8,...,17,0,0,0,1,6,0,5,0,0
B-вид спорта,223,1,0,0,0,0,0,6,0,8,...,0,0,0,0,0,6,0,3,0,0
B-видеоигра,637,0,4,0,13,0,0,1,1,19,...,58,0,0,0,0,1,0,0,0,0
B-команда,234,0,0,0,0,0,0,58,2,53,...,0,0,0,1,0,8,1,7,0,0
B-лига,120,3,0,0,0,0,0,4,0,10,...,0,0,1,0,0,3,0,0,0,0
B-локация,1172,9,0,0,0,0,0,1587,0,42,...,0,0,0,13,0,14,4,59,0,0
B-модель,354,1,15,0,2,0,0,13,21,0,...,37,0,0,0,20,7,0,15,0,0
B-название проекта,2107,0,0,0,3,0,0,34,2,903,...,7,0,0,0,0,102,4,26,0,0


In [51]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

### Посмотрим на результаты

In [52]:
text = ner_train[25]['tokens']
text = ["Как", "сделать"]

In [53]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:
            print(f'{t:15s} {label_list[idx]:10s}')
    return text, pipe(text), labels

In [54]:
predict_ner(text, tokenizer, model, pipe)

[CLS]           O         
Как             O         
сделать         O         
[SEP]           O         


(['Как', 'сделать'], [[], []], ['O', 'O', 'O', 'O'])

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [55]:
submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(text, tokenizer, model, pipe, submission):
    for i, elem in tqdm(enumerate(ner_test)):
        _, _, labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
        submission.loc[i, 'video_info'] = elem

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [56]:
result = sample_submission(text, tokenizer, model, pipe, submission)

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentiall

In [57]:
result

,video_info,entities_prediction
0,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-локация, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-видеоигра, I-видеоигра, I-видеоигра, I-видеоигра, O, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-видеоигра..."
2,NaN,"[O, O, O, O, O, O, O, O, O, O, B-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, I-Дата, I-Дата, I-Дата, O, O, O, B-название проекта, O, O, O, O, O, O, O, O, B-персона, I-персона, O, B-персона,..."
3,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, B-название проекта, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-название проекта, B-Дата, B-Дата, B-Дата, B-Дата, B-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O..."
...,...,...
1280,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1281,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, B-персона, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1282,NaN,"[O, O, O, O, O, O, O, O, B-название проекта, O, O, O, O, O, O, B-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-название проекта, O, O, B-название проекта, O, B-Дата, O, O, O, O, B-локация, O, B-Дата, I-Дата, B-Дата, O, O, O, O, O, O, O, B-персона, O, O, O, O, O, O, O, O, O, O,..."
1283,NaN,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, I-Дата, B-Дата, O, O, O, O, B-персона, I-персона, O, B-персона, I-персона, O, B-персона, I-персона, O, B-персона, I-персона, O, B-персона, I-персона, O, B-персона, I-персона, O..."


In [59]:
len(ner_test)
result.to_csv('submission.csv', index=False)